In [3]:
import os
import glob

# This code changes the YOLO bounding box labels from polygon format to cubic, (5 labels to 4 labels).
# Define input and output directories
#input_label_dir = '/Users/lewishall/Desktop/'  # Folder containing segmentation labels
#output_label_dir = '/Users/lewishall/Desktop/'  # Folder to save converted labels

# Ensure output directory exists
os.makedirs(output_label_dir, exist_ok=True)

# Get all label files
# glob.glob() searches for files that match a given pattern. And therefore 
# returns a list of filenames that match this pattern.
label_files = glob.glob(os.path.join(input_label_dir, "*.txt"))

def convert_segmentation_to_bbox(seg_label_file, det_label_file):
    with open(seg_label_file, "r") as f:
        lines = f.readlines()
    
    converted_labels = []
    
    for line in lines:
        # map applies float to every element in line.strip().split()
        # making them all floats
        # list converts all these float values into a list stored in values
        values = list(map(float, line.strip().split()))
        
        # This takes the first value in list values, converts it to an int, stores it in class_id
        class_id = int(values[0])
        
        # values[1:] takes all the numbers in values apart from the first one
        points = values[1:]  # Polygon points (x1, y1, x2, y2, ..., xn, yn)
        
        # Extract min/max coordinates for bounding box
        # Python slicing works as list[start:end:step]
        x_coords = points[0::2]  # Extract all x values
        y_coords = points[1::2]  # Extract all y values

        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)

        # Convert to YOLO format: x_center, y_center, width, height (normalized)
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min

        # Store converted label
        converted_labels.append(f"{class_id} {x_center} {y_center} {width} {height}\n")

    # Save new detection label file
    with open(det_label_file, "w") as f:
        f.writelines(converted_labels)

# Process all label files
# seg_label_file will take on the value of each segmentation label file path 
# in label_files during each iteration.
for seg_label_file in label_files:
    filename = os.path.basename(seg_label_file)
    det_label_file = os.path.join(output_label_dir, filename)
    convert_segmentation_to_bbox(seg_label_file, det_label_file)

print("Conversion complete! Check the 'detection_labels' folder.")

Conversion complete! Check the 'detection_labels' folder.
